In [86]:
load('Symmalg.sage')
n = 3
n_vars = n*(n+1)/2
var = ['s%i%i'%(i,j) for j in range(1,n+1) for i in range(1,j+1)]
var+= ['g%i%i'%(i,j) for i in range(1,n_vars+1) for j in range(1,n_vars+1)]
R = PolynomialRing(QQ, var)
R.inject_variables()

Defining s11, s12, s22, s13, s23, s33, g11, g12, g13, g14, g15, g16, g21, g22, g23, g24, g25, g26, g31, g32, g33, g34, g35, g36, g41, g42, g43, g44, g45, g46, g51, g52, g53, g54, g55, g56, g61, g62, g63, g64, g65, g66


In [87]:
gens = [s12*s13-s11*s23, s12^2-s11*s22-s13^2+s11*s33]
show(gens[0])
show(gens[1])

s12*s13 - s11*s23

s12^2 - s11*s22 - s13^2 + s11*s33

In [121]:
LieI = symmalg(gens, n=6)

Defining s11, s12, s22, s13, s23, s33
Defining s11, s12, s22, s13, s23, s33, g11, g12, g13, g14, g15, g16, g21, g22, g23, g24, g25, g26, g31, g32, g33, g34, g35, g36, g41, g42, g43, g44, g45, g46, g51, g52, g53, g54, g55, g56, g61, g62, g63, g64, g65, g66

A basis of the Lie algebra consists of the following matrices:



[
[ 1  0  0  0  0  0]  [ 0  0  0  0  0  0]  [ 0  0  0  0  0  0]
[ 0  0  0  0  0  0]  [ 1  0  0  0  0  0]  [ 0  1  0  0  0  0]
[ 0  0  0  0  0  0]  [ 0  0  0  0  0  0]  [ 0  0  0  0  0  0]
[ 0  0  0  0  0  0]  [ 0  0  0  0  0  0]  [ 0  0  0  1  0  0]
[ 0  0  0  0 -1  0]  [ 0  0  0  1  0  0]  [ 0  0  0  0  2  0]
[ 0  0  1  0  0 -1], [ 0 -2  0  0  0  0], [ 0  0 -2  0  0  2],

[ 0  0  0  0  0  0]  [0 0 0 0 0 0]  [0 0 0 0 0 0]  [0 0 0 0 0 0]
[ 0  0  0  1  0  0]  [0 0 0 0 0 0]  [0 0 0 0 0 0]  [0 0 0 0 0 0]
[ 0  0  0  0  0  0]  [1 0 0 0 0 0]  [0 1 0 0 0 0]  [0 0 1 0 0 0]
[ 0 -1  0  0  0  0]  [0 0 0 0 0 0]  [0 0 0 0 0 0]  [0 0 0 0 0 0]
[ 0  0 -1  0  0  1]  [0 0 0 0 0 0]  [0 0 0 0 0 0]  [0 0 0 0 0 0]
[ 0  0  0  0 -4  0], [1 0 0 0 0 0], [0 1 0 0 0 0], [0 0 1 0 0 0],

[0 0 0 0 0 0]  [0 0 0 0 0 0]  [0 0 0 0 0 0]  [0 0 0 0 0 0]
[0 0 0 0 0 0]  [0 0 0 0 0 0]  [0 0 0 0 0 0]  [0 0 0 0 0 0]
[0 0 0 1 0 0]  [0 0 0 0 1 0]  [0 0 0 0 0 1]  [0 0 0 0 0 0]
[0 0 0 0 0 0]  [0 0 0 0 0 0]  [0 0 0 0 0 0]  [1 0 0 0 0 0]
[0 0 0 0 0 0]  [0 0 0 0 0 0]  [0 0 0 0 0 0]  [0 1 0 0 0 0]
[0 0 0 1 0 0], [0 0 0 0 1 0], [0 0 0 0 0 1], [0 0 0 2 0 0]
]

In [122]:
LieI.Dimension()

11

In [123]:
LieI.IsLieSolvable()

true

## Solvable
Since we found out that the symmetry Lie algebra is solvable, we know(by Lie's Theorem) that the Lie algebra is upper triangularizable. In next few code blocks we'll upper triangularize the symmetry Lie algebra.

In [127]:
# Storing a basis of LieI in the list L 
L = []
for m in LieI.Basis():
    L.append(matrix(QQ,m))
show(L)

[
[ 1  0  0  0  0  0]  [ 0  0  0  0  0  0]  [ 0  0  0  0  0  0]
[ 0  0  0  0  0  0]  [ 1  0  0  0  0  0]  [ 0  1  0  0  0  0]
[ 0  0  0  0  0  0]  [ 0  0  0  0  0  0]  [ 0  0  0  0  0  0]
[ 0  0  0  0  0  0]  [ 0  0  0  0  0  0]  [ 0  0  0  1  0  0]
[ 0  0  0  0 -1  0]  [ 0  0  0  1  0  0]  [ 0  0  0  0  2  0]
[ 0  0  1  0  0 -1], [ 0 -2  0  0  0  0], [ 0  0 -2  0  0  2],

[ 0  0  0  0  0  0]  [0 0 0 0 0 0]  [0 0 0 0 0 0]  [0 0 0 0 0 0]
[ 0  0  0  1  0  0]  [0 0 0 0 0 0]  [0 0 0 0 0 0]  [0 0 0 0 0 0]
[ 0  0  0  0  0  0]  [1 0 0 0 0 0]  [0 1 0 0 0 0]  [0 0 1 0 0 0]
[ 0 -1  0  0  0  0]  [0 0 0 0 0 0]  [0 0 0 0 0 0]  [0 0 0 0 0 0]
[ 0  0 -1  0  0  1]  [0 0 0 0 0 0]  [0 0 0 0 0 0]  [0 0 0 0 0 0]
[ 0  0  0  0 -4  0], [1 0 0 0 0 0], [0 1 0 0 0 0], [0 0 1 0 0 0],

[0 0 0 0 0 0]  [0 0 0 0 0 0]  [0 0 0 0 0 0]  [0 0 0 0 0 0]
[0 0 0 0 0 0]  [0 0 0 0 0 0]  [0 0 0 0 0 0]  [0 0 0 0 0 0]
[0 0 0 1 0 0]  [0 0 0 0 1 0]  [0 0 0 0 0 1]  [0 0 0 0 0 0]
[0 0 0 0 0 0]  [0 0 0 0 0 0]  [0 0 0 0 0 0]  [1 0 0 0 0 0]
[0 0 0 0 0 0]  [0 0 0 0 0 0]  [0 0 0 0 0 0]  [0 1 0 0 0 0]
[0 0 0 1 0 0], [0 0 0 0 1 0], [0 0 0 0 0 1], [0 0 0 2 0 0]
]

In [92]:
K = QQ[sqrt(-1)]
V = VectorSpace(K,6)
espaces = []
for l in L:
    espace = []
    for S in l.right_eigenspaces():
        espace.append(V.subspace(S[1]))
    espaces.append(espace)


In [93]:
Spaces = espaces[0]
for e in espaces:
    temp = []
    for v in e:
        for s in Spaces:
            t = v.intersection(s)
            if dim(t)!=0 and t not in temp:
                temp.append(t)
    Spaces = temp
v = Spaces[0].basis()[0]
Spaces

[Vector space of degree 6 and dimension 1 over Number Field in I with defining polynomial x^2 + 1 with I = 1*I
 Basis matrix:
 [0 0 1 0 0 1]]

In [94]:
I1 = matrix.identity(6)
I1 = I1.change_ring(K)
I1[:,2] = v
t = I1[:,0]
I1[:,0] = I1[:,2]
I1[:,2] = t
show(I1)
L1 = []
for l in L:
    k = I1.inverse()*l*I1
    L1.append(k[1:,1:])
    show(k)

[0 0 1 0 0 0]
[0 1 0 0 0 0]
[1 0 0 0 0 0]
[0 0 0 1 0 0]
[0 0 0 0 1 0]
[1 0 0 0 0 1]

[ 0  0  0  0  0  0]
[ 0  0  0  0  0  0]
[ 0  0  1  0  0  0]
[ 0  0  0  0  0  0]
[ 0  0  0  0 -1  0]
[ 0  0  0  0  0 -1]

[ 0  0  0  0  0  0]
[ 0  0  1  0  0  0]
[ 0  0  0  0  0  0]
[ 0  0  0  0  0  0]
[ 0  0  0  1  0  0]
[ 0 -2  0  0  0  0]

[0 0 0 0 0 0]
[0 1 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 1 0 0]
[0 0 0 0 2 0]
[0 0 0 0 0 2]

[ 0  0  0  0  0  0]
[ 0  0  0  1  0  0]
[ 0  0  0  0  0  0]
[ 0 -1  0  0  0  0]
[ 0  0  0  0  0  1]
[ 0  0  0  0 -4  0]

...

[1 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]

[0 0 0 1 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]

[0 0 0 0 1 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]

[1 0 0 0 0 1]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]

[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 1 0 0 0]
[0 1 0 0 0 0]
[0 0 0 2 0 0]

In [95]:
V = VectorSpace(K,5)
espaces = []
for l in L1:
    espace = []
    for S in l.right_eigenspaces():
        espace.append(V.subspace(S[1]))
    espaces.append(espace)

In [96]:
Spaces = espaces[0]
for e in espaces:
    temp = []
    for v in e:
        for s in Spaces:
            t = v.intersection(s)
            if dim(t)!=0 and t not in temp:
                temp.append(t)
    Spaces = temp

In [97]:
v = Spaces[1].basis()
vm = vector(K,v[0])
I2 = matrix.identity(5)
I2 = I2.change_ring(K)
I2[:,3] = v[0]
t = I2[:,3]
I2[:,3] = I2[:,0]
I2[:,0] = t
show(I2)
L2 = []
for l in L1:
    k = I2.inverse()*l*I2
    L2.append(k[1:,1:])
    show(k)


[   0    0    0    1    0]
[   0    1    0    0    0]
[   0    0    1    0    0]
[   1    0    0    0    0]
[-2*I    0    0    0    1]

[-1  0  0  0  0]
[ 0  1  0  0  0]
[ 0  0  0  0  0]
[ 0  0  0  0  0]
[ 0  0  0  0 -1]

[  0   0   1   0   0]
[  0   0   0   0   0]
[  0   0   0   0   0]
[  0   1   0   0   0]
[  0   0 2*I  -2   0]

[2 0 0 0 0]
[0 0 0 0 0]
[0 0 1 0 0]
[0 0 0 1 0]
[0 0 0 0 2]

[-2*I    0    0    0    1]
[   0    0    0    0    0]
[   0    0    0   -1    0]
[   0    0    1    0    0]
[   0    0    0    0  2*I]

...

[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]

[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]

[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]

[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]

[  0   0   0   1   0]
[  0   0   0   0   0]
[  0   1   0   0   0]
[  0   0   0   0   0]
[  0   0   2 2*I   0]

In [98]:
V = VectorSpace(K,4)
espaces = []
for l in L2:
    espace = []
    for S in l.right_eigenspaces():
        espace.append(V.subspace(S[1]))
    espaces.append(espace)

In [99]:
Spaces = espaces[0]
for e in espaces:
    temp = []
    for v in e:
        for s in Spaces:
            t = v.intersection(s)
            if dim(t)!=0 and t not in temp:
                temp.append(t)
    Spaces = temp
v = Spaces[0].basis()[0]
v

(0, 0, 0, 1)

In [100]:
I3 = matrix.identity(4)
I3 = I3.change_ring(K)
I3[:,3] = I3[:,0]
I3[:,0] = v
L3 = []
show(I3)
for l in L2:
    k = I3.inverse()*l*I3
    L3.append(k[1:,1:])
    show(k)

[0 0 0 1]
[0 1 0 0]
[0 0 1 0]
[1 0 0 0]

[-1  0  0  0]
[ 0  0  0  0]
[ 0  0  0  0]
[ 0  0  0  1]

[  0 2*I  -2   0]
[  0   0   0   0]
[  0   0   0   1]
[  0   0   0   0]

[2 0 0 0]
[0 1 0 0]
[0 0 1 0]
[0 0 0 0]

[2*I   0   0   0]
[  0   0  -1   0]
[  0   1   0   0]
[  0   0   0   0]

...

[0 0 0 0]
[0 0 0 0]
[0 0 0 0]
[0 0 0 0]

[0 0 0 0]
[0 0 0 0]
[0 0 0 0]
[0 0 0 0]

[0 0 0 0]
[0 0 0 0]
[0 0 0 0]
[0 0 0 0]

[0 0 0 0]
[0 0 0 0]
[0 0 0 0]
[0 0 0 0]

[  0   2 2*I   0]
[  0   0   0   1]
[  0   0   0   0]
[  0   0   0   0]

In [101]:
V = VectorSpace(K,3)
espaces = []
for l in L3:
    espace = []
    for S in l.right_eigenspaces():
        espace.append(V.subspace(S[1]))
    espaces.append(espace)

In [102]:
Spaces = espaces[0]
for e in espaces:
    temp = []
    for v in e:
        for s in Spaces:
            t = v.intersection(s)
            if dim(t)!=0 and t not in temp:
                temp.append(t)
    Spaces = temp
v = Spaces[0].basis()[0]
v

(1, -I, 0)

In [103]:
I4 = matrix.identity(3)
I4 = I4.change_ring(K)
I4[:,0] = v
show(I4)
L4 = []
for l in L3:
    k = I4.inverse()*l*I4
    L4.append(k[1:,1:])
    show(k)

[ 1  0  0]
[-I  1  0]
[ 0  0  1]

[0 0 0]
[0 0 0]
[0 0 1]

[0 0 0]
[0 0 1]
[0 0 0]

[1 0 0]
[0 1 0]
[0 0 0]

[ I -1  0]
[ 0 -I  0]
[ 0  0  0]

...

[0 0 0]
[0 0 0]
[0 0 0]

[0 0 0]
[0 0 0]
[0 0 0]

[0 0 0]
[0 0 0]
[0 0 0]

[0 0 0]
[0 0 0]
[0 0 0]

[0 0 1]
[0 0 I]
[0 0 0]

In [104]:
I2_1 = matrix.identity(K,6)
I2_1[1:,1:] = I2
I3_1 = matrix.identity(K,6)
I3_1[2:,2:] = I3
I4_1 = matrix.identity(K,6)
I4_1[3:,3:] = I4
I = I1*I2_1*I3_1*I4_1
L_upper = []
for l in L:
    k = I.inverse()*l*I
    L_upper.append(k)
    show(k)

[ 0  0  0  0  0  0]
[ 0 -1  0  0  0  0]
[ 0  0 -1  0  0  0]
[ 0  0  0  0  0  0]
[ 0  0  0  0  0  0]
[ 0  0  0  0  0  1]

[  0   0   0   0   0   0]
[  0   0   0   1   0   0]
[  0   0   0 4*I  -2   0]
[  0   0   0   0   0   0]
[  0   0   0   0   0   1]
[  0   0   0   0   0   0]

[0 0 0 0 0 0]
[0 2 0 0 0 0]
[0 0 2 0 0 0]
[0 0 0 1 0 0]
[0 0 0 0 1 0]
[0 0 0 0 0 0]

[   0    0    0    0    0    0]
[   0 -2*I    1    0    0    0]
[   0    0  2*I    0    0    0]
[   0    0    0    I   -1    0]
[   0    0    0    0   -I    0]
[   0    0    0    0    0    0]

[0 0 0 0 0 1]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]

...

[1 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]

[0 0 0 1 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]

[0 1 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]

[   1 -2*I    1    0    0    0]
[   0    0    0    0    0    0]
[   0    0    0    0    0    0]
[   0    0    0    0    0    0]
[   0    0    0    0    0    0]
[   0    0    0    0    0    0]

[  0   0   0   0   0   0]
[  0   0   0  -I   1   0]
[  0   0   0   4 2*I   0]
[  0   0   0   0   0   1]
[  0   0   0   0   0   I]
[  0   0   0   0   0   0]

In [51]:
for l in L_upper:
    show(l, l.is_diagonalizable())

[ 0  0  0  0  0  0]
[ 0 -1  0  0  0  0]
[ 0  0 -1  0  0  0]
[ 0  0  0  0  0  0]
[ 0  0  0  0  0  0]
[ 0  0  0  0  0  1] True

[  0   0   0   0   0   0]
[  0   0   0   1   0   0]
[  0   0   0 4*I  -2   0]
[  0   0   0   0   0   0]
[  0   0   0   0   0   1]
[  0   0   0   0   0   0] False

[0 0 0 0 0 0]
[0 2 0 0 0 0]
[0 0 2 0 0 0]
[0 0 0 1 0 0]
[0 0 0 0 1 0]
[0 0 0 0 0 0] True

[   0    0    0    0    0    0]
[   0 -2*I    1    0    0    0]
[   0    0  2*I    0    0    0]
[   0    0    0    I   -1    0]
[   0    0    0    0   -I    0]
[   0    0    0    0    0    0] True

[0 0 0 0 0 1]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0] False

...

[1 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0] True

[0 0 0 1 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0] False

[0 1 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0] False

[   1 -2*I    1    0    0    0]
[   0    0    0    0    0    0]
[   0    0    0    0    0    0]
[   0    0    0    0    0    0]
[   0    0    0    0    0    0]
[   0    0    0    0    0    0] True

[  0   0   0   0   0   0]
[  0   0   0  -I   1   0]
[  0   0   0   4 2*I   0]
[  0   0   0   0   0   1]
[  0   0   0   0   0   I]
[  0   0   0   0   0   0] False

In [125]:
for l in L_upper:
    show(P.inverse()*l*P, l.is_diagonalizable())
    #print(latex(P.inverse()*l*P))

[-1  0  0  0  0  0]
[ 0  0  0  0  0  0]
[ 0  0  0  0  0  0]
[ 0  0  0 -1  0  0]
[ 0  0  0  0  0  0]
[ 0  0  0  0  0  1] True

[     0      1      0      0      0      0]
[     0      0      0      0      0  1/2*I]
[     0      0      0      0      0 -1/2*I]
[     0      0      1      0      0      0]
[     0      0      0      0      0      0]
[     0      0      0      0      0      0] False

[2 0 0 0 0 0]
[0 1 0 0 0 0]
[0 0 1 0 0 0]
[0 0 0 2 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0] True

[ 2*I    0    0    0    0    0]
[   0    I    0    0    0    0]
[   0    0   -I    0    0    0]
[   0    0    0 -2*I    0    0]
[   0    0    0    0    0    0]
[   0    0    0    0    0    0] True

[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 1]
[0 0 0 0 0 0] False

...

[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 1 0]
[0 0 0 0 0 0] True

[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 1 1 0 0 0]
[0 0 0 0 0 0] False

[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[1 0 0 1 0 0]
[0 0 0 0 0 0] False

[   0    0    0    0    0    0]
[   0    0    0    0    0    0]
[   0    0    0    0    0    0]
[   0    0    0    0    0    0]
[ 2*I    0    0 -2*I    1    0]
[   0    0    0    0    0    0] True

[  0  -I   0   0   0   0]
[  0   0   0   0   0 1/2]
[  0   0   0   0   0 1/2]
[  0   0   I   0   0   0]
[  0   0   0   0   0   0]
[  0   0   0   0   0   0] False

In [120]:
X = matrix(R,n_vars,1,R.gens()[:n_vars])
G = I*P
sub = {}
for i in range(6):
    sub[X[i][0].change_ring(K)] = (G*vector(X))[i]
show(sub)
p1 = (gens[0].change_ring(K))
show(p1.subs(sub))
p2 = (gens[1].change_ring(K))
show(p2.subs(sub))

{s11: s33,
 s12: (-I)*s12 + I*s22,
 s22: s23,
 s13: s12 + s22,
 s23: s11 + s13,
 s33: (2*I)*s11 + (-2*I)*s13 + s23}

(-I)*s12^2 + I*s22^2 - s11*s33 - s13*s33

-2*s12^2 - 2*s22^2 + (2*I)*s11*s33 + (-2*I)*s13*s33

In [126]:
# This is the change of basis matrtix that exposes the 4-dimensional torus in the Lie algebra as simultaneously diagonal matrices.
show(G)

[   0    0    0    0    0    1]
[   0   -I    I    0    0    0]
[   0    0    0    0    1    0]
[   0    1    1    0    0    0]
[   1    0    0    1    0    0]
[ 2*I    0    0 -2*I    1    0]

In [108]:
P2*P1
G = P2*P1
sub = {}
for i in range(6):
    sub[X[i][0].change_ring(K)] = (G*vector(X))[i]
show(sub)
p1 = (gens[0].change_ring(K))
show(p1.subs(sub))
p2 = (gens[1].change_ring(K))
show(p2.subs(sub))

{s11: s12,
 s12: s11 + s33,
 s22: (2*I)*s11 + (-2*I)*s33,
 s13: s13 + s23,
 s23: s22 + s23,
 s33: I*s13 + (-I)*s23}

-s12*s22 + s11*s13 + s11*s23 - s12*s23 + s13*s33 + s23*s33

s11^2 + (-2*I)*s11*s12 + I*s12*s13 - s13^2 + (-I)*s12*s23 - 2*s13*s23 - s23^2 + 2*s11*s33 + (2*I)*s12*s33 + s33^2